In [29]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 운영 체제 정보를 확인하고, matplotlib을 사용하여 그래프 그릴 준비하기
import platform
import matplotlib.pyplot as plt

%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Sorry')

plt.rcParams['axes.unicode_minus'] = False

In [19]:
import tensorflow as tf
with tf.device('/GPU:0'):
  a = tf.constant([[1.0, 2.0], [4.0, 5.0]])

In [20]:
print("TensorFlow version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TensorFlow version: 2.6.0
GPU is available


In [6]:
from pathlib import Path

# MNIST 데이터 세트 로드 및 분할
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
# MNIST 모델 구축 및 훈련(이미지 전처리도 처리)
tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2161 - accuracy: 0.9401 - val_loss: 0.1970 - val_accuracy:

In [6]:
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: my_mnist_model\0001\assets


INFO:tensorflow:Assets written to: my_mnist_model\0001\assets


In [5]:
from pathlib import Path

# 모델 로드 경로
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version

# 모델 로드
loaded_model = tf.keras.models.load_model(model_path, compile=False)

In [7]:
sorted([str(path) for path in model_path.parent.glob("**/*")])

['my_mnist_model\\0001',
 'my_mnist_model\\0001\\assets',
 'my_mnist_model\\0001\\fingerprint.pb',
 'my_mnist_model\\0001\\keras_metadata.pb',
 'my_mnist_model\\0001\\saved_model.pb',
 'my_mnist_model\\0001\\variables',
 'my_mnist_model\\0001\\variables\\variables.data-00000-of-00001',
 'my_mnist_model\\0001\\variables\\variables.index',
 'my_mnist_model\\0002',
 'my_mnist_model\\0002\\assets',
 'my_mnist_model\\0002\\fingerprint.pb',
 'my_mnist_model\\0002\\keras_metadata.pb',
 'my_mnist_model\\0002\\saved_model.pb',
 'my_mnist_model\\0002\\variables',
 'my_mnist_model\\0002\\variables\\variables.data-00000-of-00001',
 'my_mnist_model\\0002\\variables\\variables.index']

In [8]:
!saved_model_cli show --dir $model_path

The given SavedModel contains the following tag-sets:
'serve'


In [9]:
!saved_model_cli show --dir $model_path --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [10]:
!saved_model_cli show --dir $model_path  --tag_set serve \
                      --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_UINT8
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [11]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [10]:
# 도커 이미지 다운로드
!docker pull tensorflow/serving

Using default tag: latest
latest: Pulling from tensorflow/serving
88b83241875f: Pulling fs layer
bdf3ac5cd03d: Pulling fs layer
d9802f032d67: Pulling fs layer
6d162ca0d920: Pulling fs layer
3a1ca54df41e: Pulling fs layer
88b83241875f: Download complete
3a1ca54df41e: Download complete
bdf3ac5cd03d: Download complete
d9802f032d67: Download complete
6d162ca0d920: Download complete
Digest: sha256:1a86f4dacc3fb28abda390e303fab1bc0849683bd2bb932095a3de43cafe1659
Status: Downloaded newer image for tensorflow/serving:latest
docker.io/tensorflow/serving:latest


In [25]:
# 구글 코랩 실행 시(도커 컨테이너 실행 시에는 스킵)

%%bash --bg
tensorflow_model_server \
    --port=8500 \
    --rest_api_port=8501 \
    --model_name=my_mnist_model \
    --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

In [15]:
import json

X_new = X_test[:3] # 새로운 숫자 이미지 3개를 분류한다고 가정
request_json = json.dumps({
    'signature_name': 'serving_default',
    'instances': X_new.tolist(),
})

In [16]:
request_json

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 84, 185, 159, 151, 60, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 222, 254, 254, 254, 254, 241, 198, 198, 198, 198, 198, 198, 198, 198, 170, 52, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 67, 114, 72, 114, 163, 227, 254, 225, 254, 254, 254, 250, 229, 254, 254, 140, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 

In [17]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [18]:
y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

In [19]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]  # == "flatten_input"
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [20]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [21]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [22]:
# 새로운 MNIST 모델 버전 빌드 및 훈련
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7742 - accuracy: 0.7823 - val_loss: 0.3514 - val_accuracy: 0.9010
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3269 - accuracy: 0.9061 - val_loss: 0.2766 - val_accuracy: 0.9202
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2744 - accuracy: 0.9203 - val_loss: 0.2377 - val_accuracy: 0.9326
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2415 - accuracy: 0.9305 - val_loss: 0.2157 - val_accuracy: 0.9414
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2172 - accuracy: 0.9371 - val_loss: 0.1949 - val_accuracy: 0.9436
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1981 - accuracy: 0.9421 - val_loss: 0.1829 - val_accuracy: 0.9476
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1821 - accuracy: 0.9463 - val_loss: 0.1689 - val_accuracy:

In [23]:
model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: my_mnist_model\0002\assets


INFO:tensorflow:Assets written to: my_mnist_model\0002\assets


In [25]:
sorted([str(path) for path in model_path.parent.glob("**/*")])

['my_mnist_model\\0001',
 'my_mnist_model\\0001\\assets',
 'my_mnist_model\\0001\\fingerprint.pb',
 'my_mnist_model\\0001\\keras_metadata.pb',
 'my_mnist_model\\0001\\saved_model.pb',
 'my_mnist_model\\0001\\variables',
 'my_mnist_model\\0001\\variables\\variables.data-00000-of-00001',
 'my_mnist_model\\0001\\variables\\variables.index',
 'my_mnist_model\\0002',
 'my_mnist_model\\0002\\assets',
 'my_mnist_model\\0002\\fingerprint.pb',
 'my_mnist_model\\0002\\keras_metadata.pb',
 'my_mnist_model\\0002\\saved_model.pb',
 'my_mnist_model\\0002\\variables',
 'my_mnist_model\\0002\\variables\\variables.data-00000-of-00001',
 'my_mnist_model\\0002\\variables\\variables.index']

In [26]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"

response = requests.post(server_url, data=request_json)
response.raise_for_status()
response = response.json()

In [27]:
response.keys()

dict_keys(['predictions'])

In [28]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [26]:
import os
from google.oauth2 import service_account

project_id = "your_project_ID"  ##### 이를 프로젝트 ID로 변경. #####

# 서비스 계정 키 파일 경로 설정
key_path = "your_key_path.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

In [27]:
from google.cloud import storage

bucket_name = "hands_bucket"  ##### 고유한 버킷 이름 #####
location = "us-central1"

storage_client = storage.Client(project=project_id)
#bucket = storage_client.create_bucket(bucket_name, location=location)
bucket = storage_client.bucket(bucket_name)  # 버킷을 재사용하는 경우

In [28]:
from pathlib import Path

def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

upload_directory(bucket, "my_mnist_model")

In [ ]:
#!gsutil -m cp -r my_mnist_model gs://{bucket_name}/

In [29]:
from google.cloud import aiplatform

server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"

aiplatform.init(project=project_id, location=location)
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image,
)

Creating Model
Create Model backing LRO: projects/70292773699/locations/us-central1/models/8969012116396703744/operations/1973444598139191296
Model created. Resource name: projects/70292773699/locations/us-central1/models/8969012116396703744@1
To use this Model in another session:
model = aiplatform.Model('projects/70292773699/locations/us-central1/models/8969012116396703744@1')


In [33]:
import subprocess

def list_accelerator_types():
    try:
        result = subprocess.run(
            "gcloud compute accelerator-types list",
            capture_output=True,
            text=True,
            check=True,
            shell=True
        )
        # 명령어 출력 결과 표시
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e.stderr}")
    except FileNotFoundError as e:
        print(f"File not found error: {e}")

# 실행
list_accelerator_types()

NAME: ct3
ZONE: us-central1-a
DESCRIPTION: ct3

NAME: ct3p
ZONE: us-central1-a
DESCRIPTION: ct3p

NAME: ct5l
ZONE: us-central1-a
DESCRIPTION: ct5l

NAME: ct5lp
ZONE: us-central1-a
DESCRIPTION: ct5lp

NAME: ct5p
ZONE: us-central1-a
DESCRIPTION: ct5p

NAME: nvidia-a100-80gb
ZONE: us-central1-a
DESCRIPTION: NVIDIA A100 80GB

NAME: nvidia-h100-80gb
ZONE: us-central1-a
DESCRIPTION: NVIDIA H100 80GB

NAME: nvidia-h100-mega-80gb
ZONE: us-central1-a
DESCRIPTION: NVIDIA H100 80GB MEGA

NAME: nvidia-l4
ZONE: us-central1-a
DESCRIPTION: NVIDIA L4

NAME: nvidia-l4-vws
ZONE: us-central1-a
DESCRIPTION: NVIDIA L4 Virtual Workstation

NAME: nvidia-tesla-a100
ZONE: us-central1-a
DESCRIPTION: NVIDIA A100 40GB

NAME: nvidia-tesla-p4
ZONE: us-central1-a
DESCRIPTION: NVIDIA Tesla P4

NAME: nvidia-tesla-p4-vws
ZONE: us-central1-a
DESCRIPTION: NVIDIA Tesla P4 Virtual Workstation

NAME: nvidia-tesla-t4
ZONE: us-central1-a
DESCRIPTION: NVIDIA T4

NAME: nvidia-tesla-t4-vws
ZONE: us-central1-a
DESCRIPTION: NVIDIA

In [46]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    max_replica_count=5,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1
)

Creating Endpoint
Create Endpoint backing LRO: projects/70292773699/locations/us-central1/endpoints/4215590253055967232/operations/4065788837517852672
Endpoint created. Resource name: projects/70292773699/locations/us-central1/endpoints/4215590253055967232
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/70292773699/locations/us-central1/endpoints/4215590253055967232')
Deploying Model projects/70292773699/locations/us-central1/models/8969012116396703744 to Endpoint : projects/70292773699/locations/us-central1/endpoints/4215590253055967232


InvalidArgument: 400 Machine type "n1-standard-1" is not supported.

In [47]:
response = endpoint.predict(instances=X_new.tolist())

NameError: name 'X_new' is not defined

In [ ]:
import numpy as np

np.round(response.predictions, 2)

In [ ]:
endpoint.undeploy_all()  # 엔드포인트에서 모든 모델 배포 취소
endpoint.delete()

In [ ]:
# 버텍스 AI에서 배치 예측 작업 실행하기

batch_path = Path("my_mnist_batch")
batch_path.mkdir(exist_ok=True)
with open(batch_path / "my_mnist_batch.jsonl", "w") as jsonl_file:
    for image in X_test[:100].tolist():
        jsonl_file.write(json.dumps(image))
        jsonl_file.write("\n")

upload_directory(bucket, batch_path)

In [ ]:
batch_prediction_job = mnist_model.batch_predict(
    job_display_name="my_batch_prediction_job",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=5,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
    gcs_source=[f"gs://{bucket_name}/{batch_path.name}/my_mnist_batch.jsonl"],
    gcs_destination_prefix=f"gs://{bucket_name}/my_mnist_predictions/",
    sync=True  # 완료될 때까지 기다리지 않으려면 False로 설정.
)

In [ ]:
batch_prediction_job.output_info  #  출력 디렉토리를 표시

In [ ]:
y_probas = []
for blob in batch_prediction_job.iter_outputs():
    print(blob.name) 
    if "prediction.results" in blob.name:
        for line in blob.download_as_text().splitlines():
            y_proba = json.loads(line)["prediction"]
            y_probas.append(y_proba)

In [ ]:
y_pred = np.argmax(y_probas, axis=1)
accuracy = np.sum(y_pred == y_test[:100]) / 100

In [ ]:
accuracy

In [ ]:
mnist_model.delete()

In [ ]:
# GCS에서 만든 모든 디렉토리(즉, 디렉토리 이름의 접두사를 가진 모든 블롭)를 삭제

for prefix in ["my_mnist_model/", "my_mnist_batch/", "my_mnist_predictions/"]:
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        blob.delete()

#bucket.delete()  # 버킷 자체를 삭제하려면 주석을 제거하고 실행.
batch_prediction_job.delete()

In [ ]:
# 모바일 또는 임베디드 디바이스에 모델 배포하기
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
# 케라스 모델을 변환하는 방법
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tflite_model = converter.convert()
with open("my_converted_keras_model.tflite", "wb") as f:
    f.write(tflite_model)